# 🩺 Stage 1 — 医学因果推断入门（可运行 Notebook，Python + DoWhy）

本 Notebook 目标：
1. 用一个医疗化的玩具数据完成**从因果图到 ATE 估计**的完整流程。
2. 用 DoWhy 跑通**识别 → 估计 → 反驳/敏感性分析**的标准管线。
3. 顺带给出**PS 估计 + IPTW 的平衡性检查**代码模板，方便迁移到医学数据（MIMIC/SEER 等）。

**你需要先安装：**
```bash
pip install pandas numpy matplotlib scikit-learn statsmodels dowhy econml causalml
```

---
## 目录
1. 环境与数据加载（DoWhy 内置模拟医疗数据）
2. 因果图（DAG）构建与可识别性检查
3. ATE/CATE 估计（多方法对比：线性回归、PSM、分层、IPTW）
4. 反驳与敏感性分析（Random CC / Placebo / Subset）
5. PS + IPTW 的协变量平衡性（SMD）诊断
6. 临床解释与下一步（迁移到真实医学数据）


In [ ]:
# === 1. 环境与数据加载 ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score

import dowhy
from dowhy import CausalModel
from dowhy import datasets as dy_datasets

np.random.seed(42)
print({
    'numpy': np.__version__,
    'pandas': pd.__version__,
    'matplotlib': plt.matplotlib.__version__,
    'dowhy': dowhy.__version__
})

# 使用 DoWhy 构造一个“医学味道”的模拟数据集：
# treatment: 是否接受某药物（1/0）
# outcome y: 连续型结局，例如住院天数减少量（或某生理指标改善值）
# v0...v4: 基线协变量（年龄、合并症评分等混杂）
# z0: 工具变量（类似地理/制度差异带来的处置概率差异）
data = dy_datasets.linear_dataset(
    beta=8,
    num_common_causes=5,
    num_instruments=1,
    num_samples=1500,
    treatment_is_binary=True,
)
df = data['df'].copy()
treatment = data['treatment_name']
outcome = data['outcome_name']
common_causes = data['common_causes_names']
instruments = data['instrument_names']
print('Columns:', df.columns.tolist())
df.head()

In [ ]:
# 小型 EDA：缺失、分布、基线差异
print('Shape:', df.shape)
print('Missing counts:\n', df.isnull().sum())

fig, ax = plt.subplots()
ax.hist(df[outcome], bins=30)
ax.set_title('Outcome distribution')
ax.set_xlabel(outcome)
ax.set_ylabel('Count')
plt.show()

fig, ax = plt.subplots()
ax.boxplot([df.loc[df[treatment]==0, outcome], df.loc[df[treatment]==1, outcome]], labels=['control','treated'])
ax.set_title('Outcome by treatment group')
plt.show()


## 2. 因果图（DAG）与识别
我们基于领域知识（此处为教学示范）构造 DAG：混杂变量 $v0\dots v4$ 影响治疗与结局；工具变量 $z0$ 影响治疗但不直接影响结局。

In [ ]:
# 构造因果模型
model = CausalModel(
    data=df,
    treatment=treatment,
    outcome=outcome,
    common_causes=common_causes,
    instruments=instruments,
)
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

## 3. ATE 估计（多方法对比）
我们用四种常见方法估计平均处理效应（ATE）：
1. 线性回归（控制混杂）
2. 倾向评分匹配（PSM）
3. 倾向评分分层（Stratification）
4. 倾向评分加权（IPTW）

In [ ]:
est_lr = model.estimate_effect(
    identified_estimand,
    method_name='backdoor.linear_regression',
)
print('ATE (Linear Regression):', est_lr.value)

est_psm = model.estimate_effect(
    identified_estimand,
    method_name='backdoor.propensity_score_matching',
)
print('ATE (PSM):', est_psm.value)

est_strat = model.estimate_effect(
    identified_estimand,
    method_name='backdoor.propensity_score_stratification',
)
print('ATE (Stratification):', est_strat.value)

est_iptw = model.estimate_effect(
    identified_estimand,
    method_name='backdoor.propensity_score_weighting',
)
print('ATE (IPTW):', est_iptw.value)

## 4. 反驳与敏感性分析（Refutations）
用 DoWhy 内置 refuters 做三种快速稳健性检查：
- **Random Common Cause**：加入一个随机“混杂”，看估计是否稳定；
- **Placebo Treatment**：把处理变量打乱，看是否出现伪效应；
- **Subset Refuter**：随机抽样数据子集，看效应是否稳健。

In [ ]:
ref_random_cc = model.refute_estimate(identified_estimand, est_iptw, method_name='random_common_cause')
print(ref_random_cc)

ref_placebo = model.refute_estimate(identified_estimand, est_iptw, method_name='placebo_treatment_refuter')
print(ref_placebo)

ref_subset = model.refute_estimate(identified_estimand, est_iptw, method_name='data_subset_refuter')
print(ref_subset)

## 5. 协变量平衡性诊断（SMD）
虽然 DoWhy 会封装 PS 与权重，但日常医学研究里我们通常**单独计算 PS 与 IPTW**，并对**平衡性**做图/表检查。下面给出一个**可直接复用**的模板：

In [ ]:
def standardized_mean_difference(x_t, x_c):
    m_t, m_c = np.mean(x_t), np.mean(x_c)
    s_t, s_c = np.var(x_t, ddof=1), np.var(x_c, ddof=1)
    s_p = np.sqrt((s_t + s_c) / 2)
    return (m_t - m_c) / s_p if s_p > 0 else 0.0

X = df[common_causes].values
T = df[treatment].values

# 逻辑回归估计 PS
ps_model = make_pipeline(StandardScaler(with_mean=False), LogisticRegression(max_iter=200))
ps_model.fit(df[common_causes], T)
ps = ps_model.predict_proba(df[common_causes])[:,1]
print('PS ROC-AUC (sanity check):', roc_auc_score(T, ps))

# IPTW 权重（稳定化）
p_t = T.mean()
w = np.where(T==1, p_t/ps, (1-p_t)/(1-ps))

# 计算权重前后 SMD
smd_before = {}
smd_after = {}
for col in common_causes:
    x_t = df.loc[T==1, col].values
    x_c = df.loc[T==0, col].values
    smd_before[col] = standardized_mean_difference(x_t, x_c)

    # 加权均值/方差（用简单实现近似）
    wt_t = w[T==1]
    wt_c = w[T==0]
    xt = df.loc[T==1, col].values
    xc = df.loc[T==0, col].values
    m_t = np.average(xt, weights=wt_t)
    m_c = np.average(xc, weights=wt_c)
    v_t = np.average((xt-m_t)**2, weights=wt_t)
    v_c = np.average((xc-m_c)**2, weights=wt_c)
    s_p = np.sqrt((v_t + v_c)/2)
    smd_after[col] = (m_t - m_c) / s_p if s_p > 0 else 0.0

smd_df = pd.DataFrame({'SMD_before': smd_before, 'SMD_after_IPTW': smd_after})
smd_df

In [ ]:
fig, ax = plt.subplots()
ax.scatter(range(len(smd_df)), smd_df['SMD_before'].values, label='Before')
ax.scatter(range(len(smd_df)), smd_df['SMD_after_IPTW'].values, label='After')
ax.axhline(0.1, linestyle='--')
ax.axhline(-0.1, linestyle='--')
ax.set_xticks(range(len(smd_df)))
ax.set_xticklabels(smd_df.index, rotation=45, ha='right')
ax.set_ylabel('Standardized Mean Difference')
ax.set_title('Covariate Balance Before/After IPTW')
ax.legend()
plt.tight_layout()
plt.show()

## 6. 临床解释与下一步
- **效应量（ATE）**：建议报告估计值 + 95% 置信区间，并比较多方法结果的一致性。
- **平衡性**：SMD 
  - 常用阈值：|SMD| < 0.1 视为平衡良好；0.1~0.2 需谨慎；>0.2 需要进一步处理（重新建模/非线性 PS）。
- **迁移到真实数据**：
  1) 用 MIMIC/SEER 选取明确的**治疗**与**结局**；
  2) 依据临床知识画 DAG，选“最小充分调整集”；
  3) 重复本流程，并做好**缺失值处理**与**敏感性分析**（未测混杂）。

➡️ 下一步我可以提供一个 **MIMIC-IV 子集 + ICU 药物对 28 天死亡率的因果推断** Notebook 模板，包含生存分析（IPTW + Cox）。